In [1]:
import gc_utils  # type: ignore
import gizmo_analysis as gizmo
import h5py
import numpy as np
import utilities as ut


In [2]:
sim = "m12i"
sim_dir = "/Users/z5114326/Documents/simulations/"


proc_file = sim_dir + sim + "/" + sim + "_processed.hdf5"
proc_data = h5py.File(proc_file, "r")  # open processed data file

In [3]:
it_lst = [0, 1]
snap_lst = [214, 277, 294, 312]

# for snap in snap_lst[1:]:
#     print(snap)

In [55]:
it = 1
j = 1
past_snap = snap_lst[j]
curr_snap = snap_lst[j + 1]

it_id = gc_utils.iteration_name(it)
past_snap_id = gc_utils.snapshot_name(past_snap)
curr_snap_id = gc_utils.snapshot_name(curr_snap)

past_ptypes = proc_data[it_id]["snapshots"][past_snap_id]["ptype"][()]
past_star_mask = np.array([ptype.decode("utf-8") for ptype in past_ptypes]) == "star"

curr_ptypes = proc_data[it_id]["snapshots"][curr_snap_id]["ptype"][()]
curr_star_mask = np.array([ptype.decode("utf-8") for ptype in curr_ptypes]) == "star"

past_gcids = proc_data[it_id]["snapshots"][past_snap_id]["gc_id"][past_star_mask]
curr_gcids = proc_data[it_id]["snapshots"][curr_snap_id]["gc_id"][curr_star_mask]

common_gcids = np.intersect1d(curr_gcids, past_gcids)
len(common_gcids)

2853

In [56]:
fire_dir = sim_dir + sim + "/" + sim + "_res7100/"
past_part = gizmo.io.Read.read_snapshots("star", "index", past_snap, fire_dir, assign_pointers=True)
curr_part = gizmo.io.Read.read_snapshots("star", "index", curr_snap, fire_dir, assign_pointers=True)


# in utilities.simulation.Snapshot():
* reading:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/snapshot_times.txt

  using snapshot index = 277, redshift = 1.000


# in gizmo_analysis.gizmo_io.Read():
* reading header from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/output/snapdir_277/snapshot_277.0.hdf5
  snapshot contains the following number of particles:
    dark      (id = 1): 70514272 particles
    dark2     (id = 2): 5513331 particles
    gas       (id = 0): 65828380 particles
    star      (id = 4): 4728866 particles
    blackhole (id = 5): 0 particles

* reading the following
  species: ['star']

* reading particles from:
    snapshot_277.0.hdf5
    snapshot_277.1.hdf5
    snapshot_277.2.hdf5
    snapshot_277.3.hdf5

* reading cosmological parameters from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/initial_condition/ic_agora_m12i.conf

* checking sanity of particle properties


# in gizmo_analysis.gizmo_track.ParticleCoordinate():
  read 1 

In [57]:
past_part.Pointer.add_intermediate_pointers(curr_part.Pointer)

In [58]:
pointers = past_part.Pointer.get_pointers(
    species_name_from="star", species_names_to="star", intermediate_snapshot=True, forward=True
)

In [63]:
past_star_indices = proc_data[it_id]["snapshots"][past_snap_id]["snap_part_idx"][past_star_mask]
curr_star_indices = proc_data[it_id]["snapshots"][curr_snap_id]["snap_part_idx"][curr_star_mask]
corr_curr_star_indices = pointers[past_star_indices]

In [65]:
curr_star_indices = 0

In [64]:
i = 0
for gcid in common_gcids:
    past_idx = np.where(past_gcids == gcid)[0]
    curr_idx = np.where(curr_gcids == gcid)[0]

    if (len(past_idx) > 1) or (len(curr_idx) > 1):
        raise Warning("Multiple GC issue.")

    corr_curr_star_idx = corr_curr_star_indices[past_idx]
    curr_star_idx = curr_star_indices[curr_idx]

    # print(curr_star_idx, corr_curr_star_idx)

    if curr_star_idx != corr_curr_star_idx:
        i += 1

i

0

In [ ]:
# proc_data.close()